In [1]:
import numpy as np
import pandas as pd
import csv
import time
import sys
import json
import joblib
import os

from tqdm import tqdm

In [2]:
#csv.field_size_limit(sys.maxsize)

## Entities

- user
- artist
- album
- track
- genre

**Additional information:**
- gender
- country

#### Users

In [3]:
name = '75percent_subset'

In [4]:
path = 'data/lfm'
path_prefix = 'data/lfm/intermediate_kg'
read_path_prefix = 'data/lfm/%s' % name
write_path_prefix = 'data/lfm/intermediate_kg/%s' % name

In [5]:
# user_id, country, age, gender, creation_time
with open("%s_users.tsv" % read_path_prefix, "r", encoding='utf-8') as data, open("%s_users.txt" % write_path_prefix, "w") as f:
    datareader = csv.reader(data, quoting=csv.QUOTE_NONE)
    
    for i, row in tqdm(enumerate(datareader)):
        if i == 0:
            header = row[0].split(sep="\t")
        if i != 0:
            entry = row[0].split(sep="\t")
            f.writelines("u" + entry[0] + "\t" + entry[1] + "\t" + entry[2] + "\t" + entry[3] + "\n")
            
time.sleep(0.5)            

print(header)
print("done.")

15214it [00:00, 538644.54it/s]


['user_id', 'country', 'age', 'gender', 'creation_time']
done.


#### Artists

In [6]:
# artist_id, artist_name
#with open("data/artists.tsv", "r", encoding='utf-8') as data, open("data/kg/artists.txt", "w") as f:
#    datareader = csv.reader(data, quoting=csv.QUOTE_NONE)
#    
#    for i, row in tqdm(enumerate(datareader)):
#        if i == 0:
#            header = row[0].split(sep="\t")
#        if i != 0:
#            entry = row[0].split(sep="\t")
#            f.writelines("a" + entry[0] + "\t" + str(entry[1].encode("utf-8")) + "\n")
#            
#time.sleep(0.5)            
#
#print(header)
#print("done.")

#### Albums

In [7]:
# album_id, album_name, artist_name
#with open("data/albums.tsv", "r", encoding='utf-8') as data, open("data/kg/albums.txt", "w") as f:
#    datareader = csv.reader(data, quoting=csv.QUOTE_NONE)
#    
#    for i, row in tqdm(enumerate(datareader)):
#        if i == 0:
#            header = row[0].split(sep="\t")
#        if i != 0:
#            entry = row[0].split(sep="\t")
#            f.writelines("b" + entry[0] + "\t" + str(entry[1].encode("utf-8")) +  "\n")
#            
#time.sleep(0.5)
#
#print(header)
#print("done.")

#### Tracks

In [8]:
# track_id, artist_name, track_name
track_complete_dict = {}
counter = 0
with open("%s_tracks.tsv" % read_path_prefix, "r", encoding='utf-8') as data, open("%s_tracks.txt" % write_path_prefix, "w") as f:
    datareader = csv.reader(data, quoting=csv.QUOTE_NONE)
    
    for i, row in tqdm(enumerate(datareader)):
        if i == 0:
            header = row[0].split(sep="\t")
        if i != 0:
            entry = row[0].split(sep="\t")
            try:
                f.writelines("t" + entry[0] + "\t" + str(entry[2].encode("utf-8")) + "\n")
                track_id = "t" + entry[0]
                if track_id not in track_complete_dict:
                    track_complete_dict[track_id] = 0
            except:
                f.writelines("t" + entry[0] + "\t" + "???" + "\n")
                counter += 1
            
time.sleep(0.5)            

print(header)
print("done.")
print(counter)

3062183it [00:11, 268063.77it/s]


['track_id', 'artist', 'track']
done.
41904


In [9]:
print(len(list(track_complete_dict.keys())))

3020278


#### Genres

In [10]:
#counter = 0
#with open("data/micro_genre_tags.json", "r", encoding='utf-8') as data, open("data/kg/genres.txt", "w") as f:
#    for line in tqdm(data):
#        try:
#            entry = json.loads(line)
#            track_name = entry['_id']['track']
#            artist_name = entry['_id']['artist']
#            main_genre = list(entry['tags'].keys())[0]
#            genres = ",".join(list(entry['tags'].keys()))
#
#            f.writelines(track_name + "\t" + artist_name + "\t" + main_genre + "\t" + genres + "\n")
#        except:
#            counter += 1
#            continue
#            
#time.sleep(0.5)            
#
#print(header)
#print("done.")
#print(counter)

#### Gender & Country

In [11]:
gdict = {}
cdict = {}

gcounter = 0
ccounter = 0

with open("%s_users.txt" % write_path_prefix, "r") as data,\
open("%s/entities/%s_gender.txt"  % (path_prefix, name), "w") as f,\
open("%s/entities/%s_country.txt" % (path_prefix, name), "w") as f2:
    datareader = csv.reader(data)
    
    for i, row in tqdm(enumerate(datareader)):
        entry = row[0].split(sep="\t")
        if entry[3] == "" or entry[3] in gdict:
            gcounter += 1
        else:
            gdict[entry[3]] = 0
            f.writelines(entry[3] + "\n")
            
        if entry[1] == "" or entry[1] in cdict:
            ccounter += 1
        else:
            cdict[entry[1]] = 0
            f2.writelines(entry[1] + "\n")
            
time.sleep(0.5)
            
print("done.")
print(gcounter)
print(ccounter)

15213it [00:00, 725073.83it/s]


done.
15210
15061


#### Country

In [12]:
# DONE

## Relations

- **listened_to:** _user_ listened_to _track_
- **in_album:** _track_ in_album _album_
- **created_by:** _track_ created_by _artist_
- **has_genre**: _track_ has_genre _genre_

**Additional information:**
- **has_micro_genre**: _track_ has_micro_genre _genre_ (**or** instead of has_genre)
- **has_gender**: _user_ has_gender _gender_
- **lives_in**: _user_ lives_in _country_

In [13]:
# check genres.txt --> clean track and artist names, we filter it with that list?
# create new dict mit track names, artist names and look up for all the relations

# track_name, artist_name, main_genre, micro_genre

#track_dict = {}

#with open("data/kg/genres.txt", "r") as data: #, open("data/kg/tracks.txt", "w") as f:
#    datareader = csv.reader(data)
    
#    for i, row in tqdm(enumerate(datareader)):
#        try:
#            helper = row[0].split("\t")
#            track_name = helper[0]
#            artist_name = helper[1]
#            main_genre = helper[2]
#            micro_genre = row[1:]
#        except:
#            row = ", ".join(row)
#            helper = row.split("\t")
            
#            track_name = helper[0]
#            artist_name = helper[1]
#            main_genre = helper[2]
#            micro_genre = helper[3].split(",")[1:]
#            continue
        
#        key = track_name # + " || " + artist_name
        
#        if key not in track_dict:
#            track_dict[key] = {}
#            track_dict[key]["artist"] = artist_name
#            track_dict[key]["main_genre"] = main_genre
#            track_dict[key]["micro_genre"] = micro_genre
        
#print("saving...")

#joblib.dump(track_dict, "data/kg/track_dict.pkl")

#print("done.")

#### listened_to

In [14]:
# user_id, track_id, album_id, timestamp
counter = 0
counts_dict = {}
track_dict = {}
with open("%s_listening_events.tsv" % (read_path_prefix), "r") as data, open("%s/relations/%s_listened_to.txt" % (path_prefix, name), "w") as f:
    datareader = csv.reader(data)
    
    for i, row in tqdm(enumerate(datareader)):
        if i == 0:
            header = row[0].split(sep="\t")
        if i != 0:
            entry = row[0].split(sep="\t")
            
            track_id = "t" + entry[1]
            if track_id not in track_dict:
                track_dict[track_id] = 0
            
            key = "u" + entry[0] + ", " + "listened_to" + ", " + "t" + entry[1]
            if key not in counts_dict:
                counts_dict[key] = 0
                f.writelines("u" + entry[0] + "\t" + "listened_to" + "\t" + "t" + entry[1] + "\n")
            else:
                counts_dict[key] += 1
                
            #try:
            #    f.writelines("u" + entry[0] + "\t" + "listened_to" + "\t" + "t" + entry[1] + "\n")
            #except:
            #    counter += 1
            #    continue
            
time.sleep(0.5)            

print(header)
print("done.")
print(counter)

22763712it [01:01, 369771.32it/s]


['user_id', 'track_id', 'album_id', 'timestamp']
done.
0


In [15]:
print(len(list(track_dict.keys())))

3059566


#### created_by & has_genre & has_micro_genre & in_album

In [16]:
counter = 0

artist_dict = {}
album_dict = {}
type_dict = {}
genre_dict = {}
check_dict = {}

with open("%s/micro_genre_tags_new.json" % path, "r", encoding='utf-8') as data,\
open("%s/relations/%s_has_genre.txt" % (path_prefix, name), "w", encoding='utf-8') as f1,\
open("%s/relations/%s_has_micro_genre.txt" % (path_prefix, name), "w", encoding='utf-8') as f2,\
open("%s/relations/%s_in_album.txt" % (path_prefix, name), "w", encoding='utf-8') as f3,\
open("%s/relations/%s_created_by.txt" % (path_prefix, name), "w", encoding='utf-8') as f4,\
open("%s/entities/%s_artist.txt" % (path_prefix, name), "w", encoding='utf-8') as e1,\
open("%s/entities/%s_album.txt" % (path_prefix, name), "w", encoding='utf-8') as e2,\
open("%s/entities/%s_artist_type.txt" % (path_prefix, name), "w", encoding='utf-8') as e3,\
open("%s/entities/%s_genre.txt" % (path_prefix, name), "w", encoding='utf-8') as e4,\
open("%s/entities/%s_track.txt" % (path_prefix, name), "w", encoding='utf-8') as t: #?????????
    for line in tqdm(data):
        entry = json.loads(line)
        #print(entry)
        track_id = str(entry['_id'])
        track_name = str(entry['track']['track']) #?????????
        main_genre = str(list(entry['tags'].keys())[0])
        micro_genre = list(entry['tags'].keys())[1:]
        
        artist_id = str(entry['artist']['artist_id'])
        artist_name = str(entry['artist']['artist'])
        artist_gender = str(entry['artist']['gender'])
        artist_type = str(entry['artist']['type'])
        
        album_id = str(entry['album']['album_id'])
        album_name = str(entry['album']['album'])
        
        check = "t" + track_id
        if check in track_dict:
            if check not in check_dict: #?????????
                check_dict[check] = 0
                track_dict[check] = track_name
                t.writelines(check + "\t" + track_name + "\n")
                
            if artist_id not in artist_dict:
                artist_dict[artist_id] = 0
                e1.writelines("a" + artist_id + "\t" + artist_name + "\t" + artist_gender + "\t" + artist_type + "\n")

            if album_id not in album_dict:
                album_dict[album_id] = 0
                e2.writelines("b" + album_id + "\t" + album_name + "\n")

            if artist_type not in type_dict:
                type_dict[artist_type] = 0
                e3.writelines(artist_type + "\n")

            if main_genre not in genre_dict:
                genre_dict[main_genre] = 0
                e4.writelines(main_genre + "\n")

            for micro in micro_genre:
                m = str(micro)
                if m not in genre_dict:
                    genre_dict[m] = 0
                    e4.writelines(m + "\n")

            created_by = "t" + track_id + "\t" + "created_by" + "\t" + "a" + artist_id + "\n"
            f4.writelines(created_by)
            #print(created_by)
            has_genre = "t" + track_id + "\t" + "has_genre" + "\t" + main_genre + "\n"
            f1.writelines(has_genre)
            #print(has_genre)
            for micro in micro_genre:
                has_micro_genre = "t" + track_id + "\t" + "has_micro_genre" + "\t" + micro + "\n"
                f2.writelines(has_micro_genre)
            #    print(has_micro_genre)
            in_album = "t" + track_id + "\t" + "in_album" + "\t" + "b" + album_id + "\n"
            f3.writelines(in_album)
            #print(in_album)
        else:
            counter += 1
            
time.sleep(0.5)            

print("done.")
print(counter)

3602111it [00:36, 97957.22it/s] 


done.
2851606


In [17]:
print(len(list(check_dict.keys())))

750505


In [18]:
with open("%s/entities/%s_track_new.txt" % (path_prefix, name), "w", encoding='utf-8') as t:
    for key, value in tqdm(track_dict.items()):
        track_id = key
        track_name = value
        
        if track_name == 0:
            track_name = "???"
            
        t.writelines(track_id + "\t" + track_name + "\n")
        
time.sleep(0.5)
        
print("done.")

100%|█████████████████████████████| 3059566/3059566 [00:03<00:00, 807462.16it/s]


done.


#### in_album

In [19]:
# DONE

#### has_genre

In [20]:
# DONE

#### has_micro_genre

In [21]:
# DONE

#### has_gender & lives_in

In [22]:
# user_id, country, age, gender, creation_time
gcounter = 0
ccounter = 0
with open("%s_users.txt" % write_path_prefix, "r") as data,\
open("%s/relations/%s_has_gender.txt" % (path_prefix, name), "w") as f,\
open("%s/relations/%slives_in.txt" % (path_prefix, name), "w") as f2:
    datareader = csv.reader(data)
    
    for i, row in tqdm(enumerate(datareader)):
        entry = row[0].split(sep="\t")
        if entry[3] == "":
            gcounter += 1
        else:
            f.writelines(entry[0] + "\t" + "has_gender" + "\t" + entry[3] + "\n")
            
        if entry[1] == "":
            ccounter += 1
        else:
            f2.writelines(entry[0] + "\t" + "lives_in" + "\t" + entry[1] + "\n")
            
time.sleep(0.5)
print("done.")
print(gcounter)
print(ccounter)

15213it [00:00, 360551.87it/s]


done.
43
2760


#### lives_in

In [23]:
# DONE

# Create KG File

In [24]:
directory = '%s/relations' % path_prefix
 
with open('%s_kg.txt' % write_path_prefix, "w") as kg:
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        with open(f, "r") as rel:
            datareader = csv.reader(rel)
            for i, row in tqdm(enumerate(datareader), desc=filename):
                kg.writelines(row[0] + "\n")

75percent_subset_has_micro_genre.txt: 1864170it [00:04, 442938.02it/s]
75percent_subset_has_gender.txt: 15170it [00:00, 660249.58it/s]
75percent_subset_listened_to.txt: 10937409it [00:20, 546566.90it/s]
75percent_subsetlives_in.txt: 12453it [00:00, 697864.49it/s]
75percent_subset_created_by.txt: 750505it [00:01, 537298.15it/s]
75percent_subset_in_album.txt: 750505it [00:01, 552060.00it/s]
75percent_subset_has_genre.txt: 750505it [00:01, 533256.58it/s]


In [25]:
directory = '%s/relations' % path_prefix
 
with open('%s_kg_no_gender.txt' % write_path_prefix, "w") as kg:
    for filename in os.listdir(directory):
        if filename == 'has_gender.txt':
            continue
        f = os.path.join(directory, filename)
        with open(f, "r") as rel:
            datareader = csv.reader(rel)
            for i, row in tqdm(enumerate(datareader), desc=filename):
                kg.writelines(row[0] + "\n")

75percent_subset_has_micro_genre.txt: 1864170it [00:04, 447121.84it/s]
75percent_subset_has_gender.txt: 15170it [00:00, 662352.74it/s]
75percent_subset_listened_to.txt: 10937409it [00:20, 539587.56it/s]
75percent_subsetlives_in.txt: 12453it [00:00, 700861.02it/s]
75percent_subset_created_by.txt: 750505it [00:01, 527342.69it/s]
75percent_subset_in_album.txt: 750505it [00:01, 553098.98it/s]
75percent_subset_has_genre.txt: 750505it [00:01, 516495.46it/s]
